<a href="https://colab.research.google.com/github/mekhi-woods/HiloCATsSN1991bg/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
START UP
"""
import os
import shutil
if os.path.exists('/content/HiloCATsSN1991bg') == True:
    shutil.rmtree('/content/HiloCATsSN1991bg')
    !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git
else:
    !git clone https://github.com/mekhi-woods/HiloCATsSN1991bg.git

!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple snpy
!pip install requests


In [ ]:
"""
Plotting 91bg like 1a to get an idea of what they look like
"""
import matplotlib.pyplot as plt
import numpy as np

FILTER_WHEEL = ['u', 'g', 'r', 'i', 'B', 'V0']

if __name__ == '__main__':
    KrisciunasPath = "/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt"
    KrisciunasNames = np.genfromtxt(KrisciunasPath, dtype=str, delimiter=', ')

    allCPSPhot = "/content/HiloCATsSN1991bg/data/CSPdata/SN_photo.dat"
    allCPSPhotData = np.genfromtxt(allCPSPhot, dtype='str')

    names = allCPSPhotData[:,0]
    filters = allCPSPhotData[:,1]
    time = allCPSPhotData[:,2]
    light = allCPSPhotData[:,3]
    err = allCPSPhotData[:,4]

    plt.figure(figsize=(10,6))
    sigma = 1
    for tar in KrisciunasNames:
        for n in range(len(FILTER_WHEEL)):
            # output_names = names[(names == tar) & (filters == FILTER_WHEEL[n])]
            output_light = light[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64')
            output_time = time[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64') + 53000
            output_err = err[(names == tar) & (filters == FILTER_WHEEL[n])].astype('float64')
            plt.errorbar(output_time, output_light, yerr=output_err*sigma, fmt='o', label=FILTER_WHEEL[n])

        plt.title(tar); plt.xlabel('Time [MJD]'); plt.ylabel('Intensity [mag]')
        plt.gca().invert_yaxis()
        plt.legend()
        # plt.savefig('save\\'+str(tar)+'.png')
        plt.show()
        break


In [ ]:
"""
SNooPy fitting on CSP data
"""
import matplotlib.pyplot as plt
import numpy as np
import snpy
import os
from zipfile import ZipFile

USE_SAVED = False

def recover_dir():
    if os.path.exists('/content/saved_models') == False:
        os.mkdir('/content/saved_models')
    if os.path.exists('/content/snpy_fit_plots') == False:
        os.mkdir('/content/snpy_fit_plots')
    if os.path.exists('/content/snpy_fit_plots/snpyplots.zip'):
        os.remove('/content/snpy_fit_plots/snpyplots.zip')
    return

def snpy_fit(filePath, model='max_model', shapeParam='dm15', BandsToFit = ['B','g','r','i'], summarize=True, saveplots=False, saveLoc='/content/snpy_fit_plots/'):
    s = snpy.get_sn(filePath)

    # Set model parameters
    s.choose_model(model, stype=shapeParam)
    s.set_restbands() # Auto pick appropriate rest-bands

    # Fit data -- using David configurations
    fitargs = {'mangle':1,'calibration':0, 'quiet':False} # I don't remember what calibration is
    s.fit(BandsToFit,
          dokcorr=True,
          k_stretch=False,
          reset_kcorrs=True,
          **fitargs)

    # Show results
    if summarize:
        s.summary()
        # for param in s.parameters:
        #     print("{} = {} +/- {}".format(param, s.parameters[param], s.errors[param]))
    if saveplots:
        print('Plot saved @', saveLoc+filePath[-17:-9]+'_snpyplots')
        s.plot(outfile=saveLoc+filePath[-17:-9]+'_snpyplots')
        plt.show()

        with ZipFile('/content/snpy_fit_plots/snpyplots.zip', 'a') as zip_object:
            zip_object.write(saveLoc+filePath[-17:-9]+'_snpyplots.png')

    return s

def SNooPy2_fitting(CPSpath, tarNames, model='EBV_model2', shape='st', bands=['B','g','r','i'], output=False, snpyPlots=False):
    problemChildren = []

    if len(bands) == 0:
        bands = None

    tarPaths = []
    for tar in tarNames:
        tarPaths.append(CPSpath+'/SN'+str(tar)+'_snpy.txt')

    SNeDistances = {}
    SNeRedshifts = {}

    for i in range(len(tarPaths)):
        tarName = 'SN'+tarNames[i]
        tarSave = '/content/saved_models/'+tarName+'_'+model+'.snpy'
        print('[ '+str(i+1)+' / '+str(len(tarPaths))+'] Fiting data for '+tarName+'...')

        # Create/Retrieve Fit
        valid = True

        if os.path.exists(tarSave) and USE_SAVED:
            print(tarName, 'found in files! Pulling data...')
            s_n = snpy.get_sn(tarSave)
        elif os.path.exists(tarPaths[i]):
            try:
                s_n = snpy_fit(tarPaths[i],
                                model=model,
                                shapeParam=shape,
                                BandsToFit=None,
                                summarize=output,
                                saveplots=snpyPlots,
                                saveLoc='/content/snpy_fit_plots/') # Enter snpy fit function
            except ValueError:
                problemChildren.append(tarName)
                print('[!!!] ValueError: No data near maximum light... skipping\n')
                valid = False
            except RuntimeError:
                problemChildren.append(tarName)
                print('[!!!] RuntimeError: Model has trailed off fitting filter... skipping\n')
                valid = False
            except TypeError:
                problemChildren.append(tarName)
                print('[!!!] TypeError: m > k must hold (I have no clue what this means)... skipping\n')
                valid = False
        else:
            print(tarName, 'does not exsist in CSP data... skipping')
            valid = False

        if valid:
            # Save model
            s_n.save(tarSave)

            # Pull SNooPY distance
            snpy_mu = s_n.get_distmod() # Nab paramaters from SNe objects)

            # Update dictionary/list
            SNeDistances.update({tarName: snpy_mu})
            SNeRedshifts.update({tarName: s_n.z})

            # Print info
            print('Redshift:\t z = '+str(s_n.z))
            print('Distance: \t mu = '+str(round(snpy_mu, 4))+'\n')

    print('Problem children:\n', '[', len(problemChildren), ']', problemChildren)
    plt.clf()
    return SNeDistances, SNeRedshifts

def plot_DvD(snpyDistances, burnsDistances, size=(8,5), save=False):
    print("Ploting differences in distance calculations...")
    plt.figure(figsize=size)
    for x in snpyDistances:
        try:
            plt.scatter(burnsDistances[x], snpyDistances[x], marker='o')
            plt.text(burnsDistances[x]+0.05, snpyDistances[x]-0.05, x, fontsize='xx-small')
            print('Burns Distance:', burnsDistances[x], '| SNooPy Distance:', snpyDistances[x])
        except KeyError:
            print(x, 'not found.')
            pass
    plt.title("Burns v. SNooPy Distance")
    plt.xlabel('Burns Distance'); plt.ylabel('SNooPy Distance')
    plt.xlim(min(burnsDistances.values()), max(burnsDistances.values()))
    plt.ylim(min(burnsDistances.values()), max(burnsDistances.values()))

    if save:
        plt.savefig('burns_v_snpy_dist.png')

    plt.show()
    return

def plot_residuals(snpyDistances, burnsDistances, snpyRedshifts, size=(8,5), save=False):
    print("Ploting residuals...")
    plt.figure(figsize=size)
    for x in snpyDistances:
        try:
            plt.scatter(snpyRedshifts[x], abs(burnsDistances[x]-snpyDistances[x]), marker='o')
            plt.text(snpyRedshifts[x], abs(burnsDistances[x]-snpyDistances[x]), x, fontsize='xx-small')
            print('Redshift:', snpyRedshifts[x], '| Burns-SNooPy Distance:', burnsDistances[x]-snpyDistances[x])
        except KeyError:
            print(x, 'not found.')
            pass
    plt.title("Burns-SNooPy Residuals"); plt.xlabel('Redshift'); plt.ylabel('Burns-SNooPy')
    if save:
        plt.savefig('burns-snpy_res.png')
    plt.show()
    return

if __name__ == '__main__':
    recover_dir()

    # Initialize Data
    KrisciunasNames = np.genfromtxt("/content/HiloCATsSN1991bg/targetLists/91bglike_justnames.txt", dtype=str, delimiter=', ')

    burnsData = np.genfromtxt('/content/HiloCATsSN1991bg/targetLists/burns+25table2ext.txt', dtype=str)
    burnsDistances = {}
    burnsNames = burnsData[:, 0]
    for tar in np.stack((burnsData[:, 0], burnsData[:, 14]), axis=1):
        burnsDistances.update({'SN'+tar[0]: float(tar[1])})

    # Fitting Data
    snpyDistances, snpyRedshifts = SNooPy2_fitting(CPSpath='/content/HiloCATsSN1991bg/data/CSPdata',
                                                   tarNames=KrisciunasNames,
                                                   model='EBV_model2',
                                                   shape='st',
                                                   bands=['B','g','r','i'],
                                                   output=False,
                                                   snpyPlots=True)

    # Plot Distance v. Distance
    plot_DvD(snpyDistances, burnsDistances, size=(8,5), save=True)

    # Plot Residuals
    plot_residuals(snpyDistances, burnsDistances, snpyRedshifts, size=(8,5), save=True)


In [83]:
"""
ATLAS Data
"""
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
import requests
import os
import glob
from requests.auth import HTTPBasicAuth
from zipfile import ZipFile

API_TOKEN = "7f4e1dee8f52cf0c8cdaf55bf29d04bef4810fb4"
ATLAS_PATH = '/content/ATLAS'
N_ITER = -1 # -1 turns iteration limit off
PLOT_MODE = 'COMBINED' # SOLO or COMBINED

class ATLAS_SN():
    def __init__(self, name='Empty SN', RA=0.00, DEC=0.00,
                 t_o=np.array([0]), flux_o=np.array([0]), dflux_o=np.array([0]), mag_o=np.array([0]), dmag_o=np.array([0]),
                 t_c=np.array([0]), flux_c=np.array([0]), dflux_c=np.array([0]), mag_c=np.array([0]), dmag_c=np.array([0])):
        self.name=name
        self.RA=RA
        self.DEC=DEC

        self.t_o=t_o
        self.flux_o=flux_o
        self.dflux_o=dflux_o
        self.mag_o=mag_o
        self.dmag_o=dmag_o

        self.t_c=t_c
        self.flux_c=flux_c
        self.dflux_c=dflux_c
        self.mag_c=mag_c
        self.dmag_c=dmag_c
        return

    def __str__(self):
        return ('SN: '+self.name+' @ ('+str(self.RA)+', '+str(self.DEC)+')\n'+
                'O-Filter:'+
                '\t t [MJD]: '+str(np.min(self.t_o))+'...'+str(np.max(self.t_o))+'\n'+
                '\t\t flux [uJy]: '+str(np.min(self.flux_o))+'...'+str(np.max(self.flux_o))+'\n'+
                '\t\t dflux [duJy]: '+str(np.min(self.dflux_o))+'...'+str(np.max(self.dflux_o))+'\n'+
                'C-Filter:'+
                '\t t [MJD]: '+str(np.min(self.t_c))+'...'+str(np.max(self.t_c))+'\n'+
                '\t\t flux [uJy]: '+str(np.min(self.flux_c))+'...'+str(np.max(self.flux_c))+'\n'+
                '\t\t dflux [duJy]: '+str(np.min(self.dflux_c))+'...'+str(np.max(self.dflux_c))+'\n')

def recover_dir():
    if os.path.exists(ATLAS_PATH) == False:
        os.mkdir(ATLAS_PATH)
    if os.path.exists('/content/ATLASplots') == False:
        os.mkdir('/content/ATLASplots')
    return

def data_collection():
        if os.path.exists('/content/tmp.npz'):
            pickle = np.load('tmp.npz', allow_pickle=True)
            data = pickle['data']

        data = requests.post(
            'https://star.pst.qub.ac.uk/sne/atlas4/api/objectlist/',
            headers={'Authorization': f'Token {API_TOKEN}'},
            data={'objectlistid':2}
        ).json()

        np.savez('tmp.npz', data=data)

        count = 0
        for d in data:
            if d['observation_status'] is not None and d['observation_status'].startswith('SN Ia') and '91bg' in d['observation_status']:
                print(d['atlas_designation'],d['observation_status'].replace(' ',''),d['ra'],d['dec'])
                count += 1

                ids = d['id']
                base_url = 'https://star.pst.qub.ac.uk/sne/atlas4/lightcurveforced/1161048951013729300/'
                new_url = base_url.replace('1161048951013729300/',str(ids))
                print(new_url)

                idfile = ATLAS_PATH+'/' + str(ids)+'.txt'
                if os.path.exists(idfile):
                    continue
                urllib.request.urlretrieve(str(new_url), str(idfile))
                print(idfile)

            if count > 300:
                break

def main_processing(plot=True, quiet=False):
    if plot == False:
        PLOT_MODE = 'DISABLED'

    # [2.1] Aquire Data
    err_max = 50 # Max error before considered outlier
    ATLASfiles = glob.glob(ATLAS_PATH+'/*')
    validData_o, validData_c, validCoords = {}, {}, {}
    problemChildren = ['1032212120425304400']   # Running list of problematic SNe
                                                # '1032212120425304400' - David believes it might be a shock breakout but ATLAS reports it as SN1a
                                                #
    # [2.2] Validate Data
    for n in range(len(ATLASfiles)):
        path = ATLASfiles[n]
        name = path[15:-4]
        if quiet == False:
            print("[", n+1, "/", len(ATLASfiles), "] Slicing "+name+'...')
        data_o, data_c, coords, filters = slice_data(path, err_max, output=False)
        if data_o != None or data_c != None:
            validData_o.update({name : (data_o[0], data_o[1])})
            validData_c.update({name : (data_c[0], data_c[1])})
            validCoords.update({name : coords})
        else:
            problemChildren.append(name)
        if N_ITER != -1 and n > N_ITER:
            break
    # [2.3] Remove problem children
    print('Problem Children: ', problemChildren)
    for key in problemChildren:
        if key in validData_o:
            if quiet == False:
                print('Removing...', key)
            del validData_o[key]
            del validData_c[key]
            del validCoords[key]
        else:
            if quiet == False:
                print('Already removed...', key)
    print('\n')

    # [2.4] Saved RA & Dec
    with open('ATLASCoordinates.txt', 'w') as f:
        for tar in validCoords:
            f.write(str(validCoords[tar][0])+', '+str(validCoords[tar][1])+'\n')

    # [2.5] Plot data
    if PLOT_MODE == 'SOLO':
        if quiet == False:
            print("Plotting data [indivisual]...")
        with ZipFile('/content/ATLASplots/ATLASplots.zip', 'w') as zip_object:
            for name in validData_o:
                solo_plotting(validData_o[name][0], validData_c[name][0],
                              validData_o[name][1][:, 0], validData_c[name][1][:, 0],
                              validData_o[name][1][:, 1], validData_c[name][1][:, 1],
                              err_max, name=name, coords=validCoords[name], save=True)
                zip_object.write('/content/ATLASplots/'+name+'_ATLASplot.png')
                print(validCoords[name])

    elif PLOT_MODE == 'COMBINED':
        if quiet == False:
            print("Plotting data [combined]...")
        fig, axs = plt.subplots(len(validData_o.keys()), figsize=(12, len(validData_o.keys())*3.2))
        fig.tight_layout(pad=5.0)
        for n in range(len(validData_o.keys())):
            name = list(validData_o.keys())[n]
            combined_plotting(axs[n], validData_o[name][0], validData_c[name][0],
                              validData_o[name][1][:, 0], validData_c[name][1][:, 0],
                              validData_o[name][1][:, 1], validData_c[name][1][:, 1],
                              name=name, coords=validCoords[name])
            if N_ITER != -1 and n > N_ITER:
                break
    return validData_c, validData_o

def slice_data(path, err_max=100, output=True):
    expname = path[15:-4]
    data = np.genfromtxt(path, dtype=str, delimiter=',')
    if len(data) == 0:
        print('[!!!] File empty...skipping')
        return None, None, None, None
    else:
        t = data[:, 8].astype(float)
        uJy = np.where(data[:, 24] == 'None', np.nan, data[:, 24]).astype(float)
        uJy_err = np.where(data[:, 25] == 'None', np.nan, data[:, 25]).astype(float)
        coords = [data[0, 1].astype(float), data[0, 2].astype(float)]
        filters = data[:, 6]

    # Remove Outliers
    outliers = np.where(abs(uJy_err) > err_max)[0]
    if output:
        print('Outliers: ', outliers,'\n\t  ', uJy[outliers])
    for n in np.flip(outliers):
        t = np.delete(t, n)
        uJy = np.delete(uJy, n)
        uJy_err = np.delete(uJy_err, n)
        filters = np.delete(filters, n)

    t_o = t[np.where(filters=='o')[0]]
    t_c = t[np.where(filters=='c')[0]]
    uJy_o = np.stack((uJy[np.where(filters=='o')[0]], uJy_err[np.where(filters=='o')[0]]), axis=1)
    uJy_c = np.stack((uJy[np.where(filters=='c')[0]], uJy_err[np.where(filters=='c')[0]]), axis=1)

    data_o = [t_o, uJy_o]
    data_c = [t_c, uJy_c]

    return data_o, data_c, coords, filters

def solo_plotting(t_o, t_c, flux_o, flux_c, flux_err_o, flux_err_c, err_max, name='Empty', coords=[0, 0], size=(12, 4), save=False, saveLoc='/content/ATLASplots/'):
    plt.figure(figsize=size)

    plt.errorbar(t_o, flux_o, yerr=flux_err_o, fmt='o', color='orange')
    plt.errorbar(t_c, flux_c, yerr=flux_err_c, fmt='o', color='cyan')

    plt.title('Light Curve: '+str(name)+'\n'+str(coords[0])+', '+str(coords[1])+'\nMax Error = '+str(err_max))
    plt.xlabel('Time [MJD]')
    plt.ylabel('Flux [uJy]')
    plt.ylim(0)
    plt.savefig(saveLoc+name+'_ATLASplot.png')
    plt.show()
    return

def combined_plotting(ax, t_o, t_c, flux_o, flux_c, flux_err_o, flux_err_c, name='Empty', coords=[0, 0]):
    ax.errorbar(t_o, flux_o, yerr=flux_err_o, fmt='o', color='orange')
    ax.errorbar(t_c, flux_c, yerr=flux_err_c, fmt='o', color='cyan')

    ax.set_title('Light Curve: '+str(name)+'\n'+str(coords[0])+', '+str(coords[1]))
    ax.set_xlabel('Time [MJD]')
    ax.set_ylabel('Flux [uJy]')
    ax.set_ylim(0)
    return

def snpyASCIIFormat():
    ATLASfiles = glob.glob(ATLAS_PATH+'/*')
    problemChildren = ['1032212120425304400']

    for n in range(len(ATLASfiles)):
        name = ATLASfiles[n][15:-4]

        data = np.genfromtxt(ATLASfiles[n], dtype=str, delimiter=',')
        filters = data[:, 6]

        t=data[:, 8].astype(float)
        flux=np.where(data[:, 24] == 'None', 0, data[:, 24]).astype(float)
        dflux=np.where(data[:, 25] == 'None', 0, data[:, 25]).astype(float)

        tempSN = ATLAS_SN(name=name, RA=np.average(data[:, 1].astype(float)), DEC=np.average(data[:, 2].astype(float)),
                          t_o=t[np.where(filters=='o')[0]], flux_o=flux[np.where(filters=='o')[0]], dflux_o=dflux[np.where(filters=='o')[0]],
                          t_c=t[np.where(filters=='c')[0]], flux_c=flux[np.where(filters=='c')[0]], dflux_c=dflux[np.where(filters=='c')[0]])
        print('Before Cleaning:\n', tempSN, '\n')

        # Remove Outliers
        err_max = 100
        outliers = np.where(abs(uJy_err) > err_max)[0]
        print('Outliers: ', outliers,'\n\t  ', uJy[outliers])
        for n in np.flip(outliers):
            t = np.delete(t, n)
            uJy = np.delete(uJy, n)
            uJy_err = np.delete(uJy_err, n)
            filters = np.delete(filters, n)



        break

        # skip_header = True
        # with open(ATLASfiles[n], 'r') as f:
        #     for line in f.readlines():
        #         if skip_header:
        #             skip_header = False
        #         else:
        #             print(line)
        #             break

    # Line 1
    # Ex. SN1981D 0.005871 50.65992 -37.23272
    #     Name    Helio Z  RA        Dec

    # 'O'-filter photometry block
    # Ex. filter O
    #     674.8593      12.94   0.11
    #     Date (JD/MJD) mag     err


    # 'C'-filter photometry block


    return

if __name__ == '__main__':

    # # [1] Run first - then comment out
    # recover_dir()
    # data_collection()

    # [2] Slice data, remove outliers, & plot
    # cData, oData = main_processing(plot=False, quiet=True)

    # [3] Write ATLAS data to SNooPy ASCII file
    snpyASCIIFormat()



SN: 1173857301744945700 @ (264.7369415088767, 74.82995898557314)
O-Filter:	 t [MJD]: 59847.22153615...60019.62796205
		 flux [uJy]: -217.6330914500674...2368.2083192315035
		 dflux [duJy]: 0.0...1091.9559759880667
C-Filter:	 t [MJD]: 59845.26282965...59897.23318165
		 flux [uJy]: -8814.00047128511...205.9072908715623
		 dflux [duJy]: 0.0...863.5606396849189



In [ ]:
"""
DR3's Tmax vs. st vs EBVhost
"""
import matplotlib.pyplot as plt
import numpy as np

if __name__ == "__main__":
    data = np.genfromtxt('/content/HiloCATsSN1991bg/DR3_fits.dat', dtype=str, skip_header=1)
    DR3_st = np.stack((data[:, 1].astype(float), data[:, 2].astype(float)), axis=1)
    DR3_Tmax = np.stack((data[:, 5].astype(float), data[:, 6].astype(float)), axis=1)
    DR3_EBVhost = np.stack((data[:, 25].astype(float), data[:, 26].astype(float)), axis=1)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,5))
    fig.suptitle("DR3's Tmax vs. st vs EBVhost")
    sigmas = [[1, 1], [1, 1], [1, 1]]

    # Tmax vs. st
    ax1.errorbar(DR3_Tmax[:, 0], DR3_st[:, 0],
                 xerr=DR3_Tmax[:, 1]*sigmas[0][0], yerr=DR3_st[:, 1]*sigmas[0][1],
                 fmt='yo')
    ax1.set_xlabel('Tmax'); ax1.set_ylabel('st')
    ax1.set_title('Tmax vs. st, sigma(x='+str(sigmas[0][0])+', y='+str(sigmas[0][1])+')')

    # st vs. EBVhost
    ax2.errorbar(DR3_st[:, 0], DR3_EBVhost[:, 0],
                 xerr=DR3_st[:, 1]*sigmas[1][0], yerr=DR3_EBVhost[:, 1]*sigmas[1][1],
                 fmt='bo')
    ax2.set_xlabel('st'); ax2.set_ylabel('EBVhost')
    ax2.set_title('st vs. EBVhost, sigma(x='+str(sigmas[1][0])+', y='+str(sigmas[1][1])+')')

    # Tmax vs. EBVhost
    ax3.errorbar(DR3_Tmax[:, 0], DR3_EBVhost[:, 0],
                 xerr=DR3_Tmax[:, 1]*sigmas[2][0], yerr=DR3_EBVhost[:, 1]*sigmas[2][1],
                 fmt='ro')
    ax3.set_xlabel('Tmax'); ax3.set_ylabel('EBVhost')
    ax3.set_title('Tmax vs. EBVhost, sigma(x='+str(sigmas[2][0])+', y='+str(sigmas[2][1])+')')

    plt.show()
